In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os
import lightgbm
import matplotlib.pyplot as plt
import seaborn as sbn
import numpy as np
import pandas as pd
import warnings
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, RocCurveDisplay, accuracy_score
import shap
import itertools

In [3]:
train = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet").groupby('customer_ID').tail(4)
test = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet").groupby('customer_ID').tail(4)
train_labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
train.head()

In [4]:
#shapes
train.shape, test.shape, train_labels.shape

### Handling Missing Values

In [5]:
#Checking Missing values having more than 45  ,%
removing_columns = train.columns[(train.isna().sum()/len(train))*100>45]

#Drop the missing columns having more than 40%
train = train.drop(removing_columns, axis=1)
test = test.drop(removing_columns, axis=1)

In [6]:
# Get only the numeric values
numeric_cols = train.select_dtypes(include=[np.number])

def fill_missing_value(dataset):
    for column in numeric_cols:
        dataset[column] = dataset[column].fillna(dataset[column].median())
    return dataset

miss_handled_train = fill_missing_value(train)
miss_handled_test = fill_missing_value(test)

In [7]:
#fill the missing values
miss_handled_train = train.bfill(axis='rows').ffill(axis='rows')
miss_handled_test = test.bfill(axis='rows').ffill(axis='rows')
train = miss_handled_train
test = miss_handled_test

In [8]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [9]:
train =train.groupby('customer_ID').tail(1)
test = test.groupby('customer_ID').tail(1)

In [10]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [11]:
#shape
train.shape, train_labels.shape, test.shape

In [12]:
train = train.merge(train_labels, how='inner', on="customer_ID")

In [13]:
train = train.merge(train_labels, how='inner', on="customer_ID")

In [14]:
train.head()

In [15]:
test_data = test.copy()
train = train.drop(['index','customer_ID', 'S_2'], axis=1)
test = test.drop(['index','customer_ID', 'S_2'], axis=1)

In [16]:
#one hot Encoding for categorical features
obj_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
train = pd.get_dummies(train, columns=obj_col, drop_first=True)
test = pd.get_dummies(test, columns=obj_col, drop_first=True)
train.shape, test.shape

In [17]:
train.head()

In [18]:
Features=train.loc[:, test.columns]
target = train['target']

In [19]:
#Normalizing data Using The z-score method
def normalize_data(dataset):
    for column in dataset.columns:
        dataset[column] = (dataset[column] - dataset[column].mean()) / dataset[column].std() 
    return dataset

Features = normalize_data(Features)
test = normalize_data(test)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( Features, target, test_size=0.3, random_state=69)
Features = X_train
target = y_train

In [21]:
from lightgbm import LGBMClassifier

def LGBMModel(Feature, target):
    lgbm = LGBMClassifier(random_state=5)
    lgbm.fit(Features, target)
    return lgbm

In [22]:
from sklearn.neighbors import KNeighborsClassifier

def KNeighborsModel(Features, target):
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(Features, target)
    return neigh

In [23]:
def XGBModel(Features, target):
    xgb = XGBClassifier(n_estimators=300, max_depth=7, learning_rate=0.1).fit(Features, target)
    return xgb

In [24]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

def SVMLinearModel(Features, target):
    svmLinear = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))
    svmLinear.fit(Features, target)
    return svmLinear
    

In [26]:
# Select the final model
# Knn_model = KNeighborsModel(Features, target)
# lgbm_model = LGBMModel(Features, target)
XGB = XGBModel(Features, target)
# svmModel = SVMLinearModel(Features, target)

final_model = XGB

In [27]:
import joblib

# save
joblib.dump(final_model, "model.joblib")

In [28]:
# Cross validation
np.mean(cross_val_score(XGB, Features, target, scoring='accuracy', cv=5))

In [29]:
y_pred = final_model.predict(Features)
accuracy_score(target, y_pred)

In [30]:
cm=confusion_matrix(target, y_pred)
plt.figure(figsize=(13,5))
plt.title("Confusion Matrix")
plt.imshow(cm, alpha=0.5, cmap='PuBu')
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center")
plt.show()

In [32]:
print(classification_report(target, y_pred))

In [34]:
test_data['prediction']=final_model.predict_proba(test)[:,1]
# test_data['prediction']=final_model._predict_proba_lr(test)[:,1]
test_data[['customer_ID','prediction']].to_csv("xgb.csv", index=False)